<a href="https://colab.research.google.com/github/Srikanth9142/ML-works/blob/master/Spectograms_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

Using TensorFlow backend.


In [0]:

train = pd.read_csv('/content/drive/My Drive/spectograms/spectograms/train_data/train.csv')


In [4]:
train.shape

(2750, 2)

In [5]:
train_image=[]
for i in tqdm(range(2750)):
    img = image.load_img('/content/drive/My Drive/spectograms/spectograms/train_data/images/'+train['language'][i],target_size=(224,224,1),grayscale=False)
    img = image.img_to_array(img)
    img = img/225
    train_image.append(img)

100%|██████████| 2750/2750 [22:15<00:00,  2.35it/s]


In [0]:
X_train = np.array(train_image)

In [7]:
type(X_train)

numpy.ndarray

In [8]:
X_train.shape



(2750, 224, 224, 3)

In [9]:
y_trainM = np.zeros((2750,11))
iter = 0
for i in train['id'].values:
  index = int(i)
  y_trainM[iter][index-1] = 1
  iter += 1
print(y_trainM.shape)

(2750, 11)


In [0]:

# y_train=train['id'].values
# y_train = to_categorical(y_train)


In [0]:
test = pd.read_csv('/content/drive/My Drive/spectograms/spectograms/test_data/test.csv')

In [11]:
test.shape

(1122, 2)

In [12]:
test_image=[]
for i in tqdm(range(test.shape[0])):
    img = image.load_img('/content/drive/My Drive/spectograms/spectograms/test_data/images/'+test['language'][i],target_size=(224,224,1),grayscale=False)
    img = image.img_to_array(img)
    img = img/225
    test_image.append(img)

100%|██████████| 1122/1122 [08:48<00:00,  2.48it/s]


In [0]:
X_test = np.asarray(test_image)

In [14]:

type(X_test)

numpy.ndarray

In [15]:
X_test.shape

(1122, 224, 224, 3)

In [16]:
y_testM = np.zeros((1122,11))
iter = 0
for i in test['id'].values:
  index = int(i)
  y_testM[iter][index-1] = 1
  iter += 1
print(y_testM.shape)

(1122, 11)


In [0]:
y_test = y_testM
y_train = y_trainM

In [0]:
 
from keras.optimizers import Adam
from keras import applications
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dense, Dropout, Activation, Flatten, Add, BatchNormalization,AveragePooling2D
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import GlobalAveragePooling2D,Dropout,Dense
from keras.models import Model

In [19]:
base_model=applications.resnet50.ResNet50(weights=None,include_top=False,input_shape = (224,224,3))
x = base_model.output
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = AveragePooling2D(pool_size=2)(x)
z1 = Flatten()(x)
z1 = Dropout(0.5)(z1)
outputs = Dense(11,activation='softmax',kernel_initializer='he_normal')(z1)
rnet=Model(inputs=base_model.input,outputs=outputs)
adam=Adam(lr=0.001)
rnet.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])















Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [20]:
rnet.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [21]:
rnet.fit(X_train,y_train,epochs=25,batch_size=50)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/25
2750/2750 [==============================] - 91s 33ms/step - loss: 2.9031 - acc: 0.1796
Epoch 2/25
2750/2750 [==============================] - 72s 26ms/step - loss: 2.6231 - acc: 0.2596
Epoch 3/25
2750/2750 [==============================] - 72s 26ms/step - loss: 2.3673 - acc: 0.3418
Epoch 4/25
2750/2750 [==============================] - 72s 26ms/step - loss: 2.1288 - acc: 0.4153
Epoch 5/25
2750/2750 [==============================] - 72s 26ms/step - loss: 1.8951 - acc: 0.4665
Epoch 6/25
2750/2750 [==============================] - 72s 26ms/step - loss: 1.2505 - acc: 0.6658
Epoch 7/25
2750/2750 [==============================] - 72s 26ms/step - loss: 0.8850 - acc: 0.7560
Epoch 8/25
2750/2750 [==============================] - 72s 26ms/step - loss: 0.9372 - acc: 0.8015
Epoch 9/25
2750/2750 [==============================] - 72s 26ms/step - loss: 0.6809 - acc: 0.8942
Epoch 10/25
2

In [22]:
rnet.evaluate(X_test,y_test)

1122/1122 [==============================] - 12s 11ms/step


[8.21878604922915, 0.09714795008912656]

In [0]:
result = rnet.predict(X_test)

In [24]:
len(result[0])

11

In [0]:
def prediction(targ):
    accents = ['bengali','dogri','gujarathi','hindi','malayalam','manipuri','marathi','odisha','tamil','telugu','urdu']
    np.set_printoptions(formatter={'float_kind':'{:.1f}'.format})
    #np.set_printoptions(formatter={'float_kind':'{:.1f}'.format})
    l=[]
    out_len = len(targ)
    in_len = len(targ[0])
    for i in range(in_len):
        sumi = 0
        for j in range(out_len):
            sumi+=targ[j][i]
        l.append(sumi)
    print(l)
    maximum = -1
    max_ind = -1
    for i in range(len(l)):
        if(l[i] > maximum):
            maximum = l[i]
            max_ind = i+1
    # print(maximum, max_ind)
    print("Accent: ",accents[max_ind-1])
    for i in l:
        if i!=0:
            per = (i/float(out_len))*100
            if(per>1):
                print("{1:.2f}% -> {0}".format(accents[l.index(i)],per))

In [26]:
print(result.shape)

(1122, 11)


In [27]:
len(result[1019:1121])

102

In [28]:
prediction(result)

[54.444928977251486, 111.69436364977653, 51.86128294313463, 43.7109302835544, 109.77651671484708, 131.76353776380768, 95.42932015560687, 156.55644551886115, 128.97246635153837, 139.5761924572192, 98.21401789164531]
Accent:  odisha
4.85% -> bengali
9.95% -> dogri
4.62% -> gujarathi
3.90% -> hindi
9.78% -> malayalam
11.74% -> manipuri
8.51% -> marathi
13.95% -> odisha
11.49% -> tamil
12.44% -> telugu
8.75% -> urdu


Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_12[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

Train on 2750 samples, validate on 1122 samples
Epoch 1/25
2750/2750 [==============================] - 70s 26ms/step - loss: 1.5977 - acc: 0.4862 - val_loss: 3.3317 - val_acc: 0.0740
Epoch 2/25
2750/2750 [==============================] - 40s 15ms/step - loss: 0.1418 - acc: 0.9771 - val_loss: 3.4620 - val_acc: 0.1159
Epoch 3/25
2750/2750 [==============================] - 40s 15ms/step - loss: 0.0202 - acc: 0.9996 - val_loss: 3.8669 - val_acc: 0.0954
Epoch 4/25
2750/2750 [==============================] - 40s 14ms/step - loss: 0.0081 - acc: 1.0000 - val_loss: 3.8236 - val_acc: 0.1016
Epoch 5/25
2750/2750 [==============================] - 40s 15ms/step - loss: 0.0049 - acc: 1.0000 - val_loss: 3.8854 - val_acc: 0.0963
Epoch 6/25
2750/2750 [==============================] - 40s 15ms/step - loss: 0.0038 - acc: 1.0000 - val_loss: 3.9907 - val_acc: 0.0980
Epoch 7/25
2750/2750 [==============================] - 40s 15ms/step - loss: 0.0024 - acc: 1.0000 - val_loss: 4.0414 - val_acc: 0.0954


[37.34150474012783, 115.75505474405512, 21.865092207240878, 72.89690671249309, 124.98740390902714, 83.89503789060221, 127.479214588865, 103.11245531284901, 143.08579483447352, 115.29568962082175, 176.2858460666519]
Accent:  urdu
3.33% -> bengali
10.32% -> dogri
1.95% -> gujarathi
6.50% -> hindi
11.14% -> malayalam
7.48% -> manipuri
11.36% -> marathi
9.19% -> odisha
12.75% -> tamil
10.28% -> telugu
15.71% -> urdu
